In [1]:
import cv2
import numpy as np
import os
import pickle

In [30]:
with open('processed_images.pickle', 'rb') as f:
    processed_images = pickle.load(f)
processed_images, len(processed_images)

(['test2.jpeg', 'test3.jpeg'], 2)

In [31]:
with open('window_labels.pickle', 'rb') as f:
    window_labels = pickle.load(f)
window_labels

        

{'0_740_0_test2.jpeg': 0,
 '0_750_0_test2.jpeg': 0,
 '0_760_0_test2.jpeg': 0,
 '0_770_0_test2.jpeg': 0,
 '10_730_0_test2.jpeg': 0,
 '10_740_0_test2.jpeg': 0,
 '10_750_0_test2.jpeg': 0,
 '10_760_0_test2.jpeg': 0,
 '20_730_0_test2.jpeg': 0,
 '20_740_0_test2.jpeg': 0,
 '20_750_0_test2.jpeg': 0,
 '20_760_0_test2.jpeg': 0,
 '30_720_0_test2.jpeg': 0,
 '30_730_0_test2.jpeg': 0,
 '30_740_0_test2.jpeg': 0,
 '30_750_0_test2.jpeg': 0,
 '40_720_0_test2.jpeg': 0,
 '40_730_0_test2.jpeg': 0,
 '40_740_0_test2.jpeg': 0,
 '40_750_0_test2.jpeg': 0,
 '50_710_0_test2.jpeg': 0,
 '50_720_0_test2.jpeg': 0,
 '50_730_0_test2.jpeg': 0,
 '50_740_0_test2.jpeg': 0,
 '60_710_0_test2.jpeg': 0,
 '60_720_0_test2.jpeg': 0,
 '60_730_0_test2.jpeg': 0,
 '60_740_0_test2.jpeg': 0,
 '70_700_0_test2.jpeg': 0,
 '70_710_0_test2.jpeg': 0,
 '70_720_0_test2.jpeg': 0,
 '70_730_0_test2.jpeg': 0,
 '80_700_0_test2.jpeg': 0,
 '80_710_0_test2.jpeg': 0,
 '80_720_0_test2.jpeg': 0,
 '80_730_0_test2.jpeg': 0,
 '90_690_0_test2.jpeg': 0,
 '90_

In [30]:
def load_images_from_folder(folder):
    image_tuples = []
    for filename in os.listdir(folder):
        if (filename not in processed_images):
            img = cv2.imread(os.path.join(folder,filename))
            if (img is not None):
                image_tuples.append((filename, img))
    return image_tuples

In [31]:
# MinMax Scaler
def minmaxScaler(image):
    minPixel = np.amin(image)
    maxPixel = np.amax(image)
    image = (image - minPixel) / (maxPixel - minPixel)
    return image

In [32]:
def threshold_image(image, threshold=0.1):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = minmaxScaler(gray)
    for i in range(len(gray)):
        for j in range(len(gray[i])):
            gray[i][j] = int((gray[i][j] > threshold) and not (image[i][j][0] < 255 * (1 - threshold) and image[i][j][1] < 255 * (1 - threshold) and image[i][j][2] > 255 * threshold)) * 255
    return gray

In [34]:
def generate_windows(image, filename, window_size, interior_size, increment, folder):
    gray_img = threshold_image(image)
    x = 0
    while (x < image.shape[0] - window_size):
        row_has_intersection = False
        y = 0
        while (y < image.shape[1] - window_size):
            window = image[x : x + window_size, y : y + window_size, :]
            has_intersection = False
            
            gray = gray_img[x : x + window_size, y : y + window_size]
            sum_of_grayscale_pixels = np.sum(gray) / 255
            
            # if the image is not completely white, we can add it to the dataset
            if (sum_of_grayscale_pixels < window_size * window_size) and sum_of_grayscale_pixels >= window_size * window_size/10:
                
                # check center interior square for any red pixels
                for x_interior in range (int(window_size/2 - interior_size/2), int(window_size/2 + interior_size/2)):
                    for y_interior in range (int(window_size/2 - interior_size/2), int(window_size/2 + interior_size/2)): 
                        # check if pixel is red
                        if (not has_intersection and 
                            window[x_interior][y_interior][0] < 100 and 
                            window[x_interior][y_interior][1] < 100 and
                            window[x_interior][y_interior][2] > 200):
                            row_has_intersection = True
                            has_intersection = True

                window_id = str(x) + "_" + str(y) + "_" + str(int(has_intersection)) + "_" + filename
                window_labels[window_id] = int(has_intersection)
                cv2.imwrite(os.path.join(folder, window_id), gray)
                
            if (has_intersection):
                y += int(increment / 5)
            else:
                y += increment
        if (row_has_intersection):
            x += int(increment / 5)
        else:
            x += increment

    processed_images.append(filename)

In [39]:
image_tuples = load_images_from_folder("test_images")
for image_tuple in image_tuples:
    filename = image_tuple[0]
    image = image_tuple[1]
    generate_windows(image=image, filename=filename, window_size=25, interior_size=5, increment=2, folder="test_mini_images")

In [38]:
with open('processed_images.pickle', 'wb') as f:
    pickle.dump(processed_images, f)

In [39]:
with open('window_labels.pickle', 'wb') as f:
    pickle.dump(window_labels, f)

In [38]:
processed_images = []
window_labels = {}

In [36]:
window_labels

{'108_318_0_test1.jpeg': 0,
 '108_320_0_test1.jpeg': 0,
 '108_322_0_test1.jpeg': 0,
 '108_324_0_test1.jpeg': 0,
 '108_326_0_test1.jpeg': 0,
 '108_328_0_test1.jpeg': 0,
 '108_330_0_test1.jpeg': 0,
 '108_332_0_test1.jpeg': 0,
 '108_334_0_test1.jpeg': 0,
 '108_336_0_test1.jpeg': 0,
 '108_338_0_test1.jpeg': 0,
 '108_340_0_test1.jpeg': 0,
 '108_342_0_test1.jpeg': 0,
 '108_344_0_test1.jpeg': 0,
 '108_346_0_test1.jpeg': 0,
 '110_318_0_test1.jpeg': 0,
 '110_320_0_test1.jpeg': 0,
 '110_322_0_test1.jpeg': 0,
 '110_324_0_test1.jpeg': 0,
 '110_326_0_test1.jpeg': 0,
 '110_328_0_test1.jpeg': 0,
 '110_330_0_test1.jpeg': 0,
 '110_332_0_test1.jpeg': 0,
 '110_334_0_test1.jpeg': 0,
 '110_336_0_test1.jpeg': 0,
 '110_338_0_test1.jpeg': 0,
 '110_340_0_test1.jpeg': 0,
 '110_342_0_test1.jpeg': 0,
 '110_344_0_test1.jpeg': 0,
 '110_346_0_test1.jpeg': 0,
 '110_348_0_test1.jpeg': 0,
 '112_316_0_test1.jpeg': 0,
 '112_318_0_test1.jpeg': 0,
 '112_320_0_test1.jpeg': 0,
 '112_322_0_test1.jpeg': 0,
 '112_324_0_test1.jp

In [37]:
print(len(processed_images))
processed_images

1


['test1.jpeg']

In [40]:
import shutil
def new_threshold(gray, threshold=25):
    for i in range(len(gray)):
        for j in range(len(gray[i])):
            gray[i][j] = int(gray[i][j] > threshold) * 255
    return gray
directory = "test_mini_images"
# new_directory = "test"
counter = 0
new_counter = 0
for filename in os.listdir(directory):
#     print(filename, directory, os.path.join(directory, filename))
#     if counter < 100:
#         print(filename)
#     if counter > 30000:
#         break
    img = cv2.imread(os.path.join(directory,filename), 0)
    img = new_threshold(img)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    counter += 1
    if np.sum(img)/255 < 60:
        new_counter += 1
#         shutil.move(os.path.join(directory, filename), os.path.join(new_directory, filename))

print(counter, new_counter)
        


129405 0
